In [1]:
import pandas as pd
import os
import json
import openai
import config
import numpy as np
openai.organization = config.openai_organization
openai.api_key = config.openai_api_key
df = pd.read_csv("gen3_chatgpt_pron.csv")

In [2]:
os.chdir('./chatgpt_prompts/')

with open('verify_system.txt', 'r') as msg_system:
    msg_system = msg_system.read()

with open('verify_user.json', 'r') as msg_user:
    msg_user = msg_user.read()

os.chdir("../")

In [11]:
def generate(row):
    # if not row['well_formed']:
    #     return row
    # if row['is_literal']:
    #     return row
    if not row['error']:
        return row
    row['error'] = False
    try:
        return generate_helper(row)
    except:
        row['error'] = True
        return row

def generate_helper(row):
    messages = [
        {
            "role": "system",
            "content": msg_system,
        },
        {
            "role": "user",
            "content": msg_user \
                .replace("IDIOM", row['idiom']) \
                .replace("HYPOTHESIS", row['intended_literal'])
        },
    ]

    text = openai.ChatCompletion.create(
        model="gpt-4",
        messages=messages,
        max_tokens=512,
        temperature=0, # the higher this value, the less deterministic
        top_p=1, # the higher this value, the wider range of vocab is used
    ).choices[0].message.content.strip()

    response = json.loads(text)
    row['is_literal'] = response['is_literal']
    row['bad_idiom'] = response['bad_idiom']
    row['justification'] = response['justification'].strip()
    return row

In [12]:
from tqdm import tqdm
tqdm.pandas()
result = result \
    .progress_apply(generate, axis=1)

100%|██████████| 717/717 [00:00<00:00, 2858.92it/s]


In [5]:
result.to_csv("verif3_chatgpt.csv", index=False)

In [7]:
result[~result['is_literal']]

,idiom,meaning,intended_ambiguous,intended_figurative,intended_literal,is_literal,justification,converted,iter,bad_idiom,...,intended_figurative_old,intended_literal_old,maxvote - for sort,maxvote_ambiguous,maxvote_figurative,maxvote_literal,well_formed,annotations_ambiguous,annotations_figurative,annotations_literal
17,a whale of,very big (piece) of something,had a whale of a time,"at the amusement park, we had a whale of a tim...",the marine biologist had a whale of a time stu...,False,The idiom 'a whale of' is used figuratively to...,False,0,False,...,She had a whale of a time at the amusement park,She had a whale of a time cutting up the whale,figurative : figurative : figurative,figurative,figurative,figurative,False,"['figurative', 'figurative', 'figurative']","['ambiguous', 'figurative', 'figurative']","['figurative', 'ambiguous', 'figurative']"
24,act of God,an occurrence of out of control natural forces...,an act of God occurred,the sudden storm that saved the village was an...,"in the play, the moment when an act of God occ...",False,The idiom 'act of God' usually refers to a nat...,False,0,False,...,It was an act of God when the hurricane destro...,It was an act of God when the choir sang to pr...,figurative : figurative : ambiguous,figurative,figurative,ambiguous,False,"['figurative', 'figurative', 'ambiguous']","['figurative', 'figurative', 'figurative']","['literal', 'ambiguous', 'ambiguous']"
33,all bark and no bite,"threatening, aggressive, but not willing to en...",seemed all bark and no bite,the aggressive manager seemed all bark and no ...,"the dog, which seemed all bark and no bite, ne...",False,The idiom 'all bark and no bite' is used figur...,False,0,False,...,He is all bark and no bite and is too afraid t...,He is all bark and no bite as he only barks an...,ambiguous : figurative : literal,ambiguous,figurative,literal,True,"['ambiguous', 'ambiguous', 'ambiguous']","['figurative', 'figurative', 'figurative']","['ambiguous', 'literal', 'literal']"
37,all the rage,when something is the height of popularity,that fashion trend is all the rage,"these days, that fashion trend is all the rage...","at the angry mob, that fashion trend is all th...",False,The idiom 'all the rage' is used figuratively ...,False,0,False,...,It's all the rage these days to wear bright sh...,It's all the rage in the conference room becau...,ambiguous : figurative : figurative,ambiguous,figurative,figurative,False,"['figurative', 'ambiguous', 'ambiguous']","['figurative', 'figurative', 'figurative']","['figurative', 'figurative', 'figurative']"
39,an arm and a leg,a large amount of money,cost an arm and a leg,"the luxurious vacation cost an arm and a leg, ...",the prosthetic limbs cost an arm and a leg at ...,False,The idiom 'an arm and a leg' is used figurativ...,False,0,False,...,The cost of the restaurant was an arm and a le...,"""The cost of the restaurant was an arm and a ...",figurative : figurative : figurative,figurative,figurative,figurative,False,"['figurative', 'figurative', 'figurative']","['figurative', 'figurative', 'figurative']","['ambiguous', 'figurative', 'figurative']"
71,bad blood,to have ill feelings towards someone,there's bad blood between them,"since the argument, there's bad blood between ...","due to a rare genetic disorder, there's bad bl...",False,Although the hypothesis mentions a genetic dis...,False,0,False,...,There is bad blood between them because of the...,There is bad blood between them since their la...,ambiguous : figurative : figurative,ambiguous,figurative,figurative,False,"['ambiguous', 'ambiguous', 'figurative']","['figurative', 'figurative', 'figurative']","['figurative', 'figurative', 'figurative']"
124,"blood, sweat, and tears",putting a lot of effort to get a task done.,"put blood, sweat, and tears into it","they put blood, sweat, and tears into it to ac...",after a long day of manual labor and a minor i...,False,Although the hypothesis mentions manual labor ...,False,0,False,...,"It took b

In [18]:
after = pd.read_csv("verif3_chatgpt.csv")
before = pd.read_csv("verif1_chatgpt.csv")
df = pd.merge(before, after, on='idiom', how='inner')
diff = df[df['is_literal_x'] != df['is_literal_y']].reset_index(drop=True)
diff.to_csv("diff.csv", index=False)

In [19]:
res1 = pd.read_csv("gen1_chatgpt.csv")
res2 = pd.read_csv("gen3_chatgpt.csv")

In [24]:
pd.merge(res1, res2[['idiom']], on='idiom', how='inner').to_csv("gen_final_chatgpt_prePronounConversion.csv", index=False)

In [23]:
res2

,idiom,meaning,intended_ambiguous,intended_figurative,intended_literal,is_literal,justification,converted,iter,bad_idiom,...,intended_figurative_old,intended_literal_old,maxvote - for sort,maxvote_ambiguous,maxvote_figurative,maxvote_literal,well_formed,annotations_ambiguous,annotations_figurative,annotations_literal
0,a bed of roses,easy option,like a bed of roses,her new job was like a bed of roses compared t...,the garden was designed like a bed of roses wi...,True,"In this context, the phrase 'a bed of roses' i...",False,0,False,...,It's not a bed of roses trying to rebuild our ...,It's not a bed of roses because we used cotton...,ambiguous : figurative : literal,ambiguous,figurative,literal,True,"['ambiguous', 'ambiguous', 'ambiguous']","['figurative', 'figurative', 'figurative']","['literal', 'literal', 'literal']"
1,a bird in the hand,a possession that is worthy,a bird in the hand,"he decided to accept the job offer, believing ...","holding a bird in the hand, she carefully exam...",True,The hypothesis describes a situation where som...,False,0,False,...,There's a bird in his hand because he's consid...,There's a bird in his hand because he caught it,ambiguous : figurative : literal,ambiguous,figurative,literal,True,"['ambiguous', 'ambiguous', 'ambiguous']","['figurative', 'figurative', 'figurative']","['literal', 'literal', 'literal']"
2,a cuckoo in the nest,an unwelcome intruder in a place or situation.,found a cuckoo in the nest,when the new employee started causing problems...,"while observing the bird's nest, the ornitholo...",True,The hypothesis is a literal interpretation of ...,False,0,False,...,He was a cuckoo in the nest because he had dif...,He was a cuckoo in the nest as he sneaked his ...,ambiguous : figurative : figurative,ambiguous,figurative,figurative,False,"['ambiguous', 'ambiguous', 'ambiguous']","['figurative', 'figurative', 'figurative']","['figurative', 'figurative', 'figurative']"
3,a damp squib,an event which people think will be exciting b...,<turned out to be a damp squib>,the highly anticipated party <turned out to be...,"the firework, soaked in water, <turned out to ...",False,The idiom 'a damp squib' is used figuratively ...,False,0,False,...,The event was a damp squib as it failed to liv...,The event was a damp squib because it was cons...,figurative : figurative : ambiguous,figurative,figurative,ambiguous,False,"['figurative', 'figurative', 'figurative']","['ambiguous', 'figurative', 'figurative']","['figurative', 'ambiguous', 'ambiguous']"
4,a good man is hard to find,This proverb refers to women who are looking f...,a good man is hard to find,"in today's world, a good man is hard to find f...","in the crowded shopping mall, a good man is ha...",True,"In this context, the idiom is used literally a...",False,0,False,...,It's a good man that's hard to find nowadays s...,It's a good man that's hard to find since he's...,ambiguous : figurative : literal,ambiguous,figurative,literal,True,"['literal', 'ambiguous', 'ambiguous']","['figurative', 'figurative', 'literal']","['literal', 'literal', 'literal']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,you reap what you sow,"if you do bad things to people, bad things wil...",reap what you sow,"if you treat people poorly, you'll reap what y...","after months of hard work in the garden, it's ...",True,"In this context, the idiom 'reap what you sow'...",False,0,False,...,"You reap what you sow, so work hard and the re...","You reap what you sow, so you'll need to get o...",ambiguous : figurative : literal,ambiguous,figurative,literal,True,"['ambiguous', 'ambiguous', 'figurative']","['ambiguous', 'figurative', 'figurative']","['literal', 'literal', 'literal']"
713,Young Turk,a young person who has many new ideas and want...,a Young Turk in the company,he quickly became a Young Turk in the company ...,the new employee was a Young Turk in the compa...,True,"In this case, the hypothesis is a literal inte...",